# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [1]:
pip install --upgrade opencv-python

In [2]:
pip install opencv-python

In [3]:
!pip install ultralytics

In [4]:
from google.colab import drive
import cv2
from ultralytics import YOLO

In [5]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [6]:
model = YOLO('/content/drive/MyDrive/YOLODataset/YOLO_Model.pt')

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [7]:
input_video = '/content/drive/MyDrive/YOLODataset/YOLO_Video.mp4'
capture = cv2.VideoCapture(input_video)

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [8]:
height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [9]:
width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))

In [10]:
fps = int(capture.get(cv2.CAP_PROP_FPS))

## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [11]:
output_video = 'output_video.mp4'
codec = cv2.VideoWriter_fourcc(*'mp4v')

In [12]:
output = cv2.VideoWriter(output_video, codec, fps, (width, height))

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [13]:
lw=width//2
lh= height//2

In [14]:
lc= 0
rc =0
bc=0
tc=0

In [15]:
green = (0, 255, 0)
black = (0,0,0)
red=(0, 0, 255)
size = 0.5
font = cv2.FONT_HERSHEY_SIMPLEX
t=3

In [16]:
while capture.isOpened():
    re, frame = capture.read()
    if not re:
        break

    final_output = model(frame)

    for box in final_output[0].boxes:
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
        confidence_score = box.conf[0].cpu().numpy()
        class_ID = int(box.cls[0].cpu().numpy())

        if class_ID == 2:
            label = f'Car: {confidence_score:.2f}'
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), green, t)
            cv2.putText(frame, label, (int(x1), int(y1) - 10), font, size, green, t)


            cx = (x1 + x2) / 2
            cy = (y1 + y2) / 2

            if cx < lw:
                lc += 1
            else:
                rc += 1

            if cy < lh:
                tc += 1
            else:
                bc += 1

    cv2.line(frame, (lw, 0), (lw, frame.shape[0]), red, t)
    cv2.line(frame, (0, lh), (frame.shape[1], lh), red, t)
    cv2.putText(frame, f'Reem Alyabis', (10, 20), font, size, green, t)
    cv2.putText(frame, f'Left_Top Counter: {lc}', (10, 30), font, size, black, t)
    cv2.putText(frame, f'Right_Top Counter: {rc}', (lw + 10, 30), font, size, black, t)
    cv2.putText(frame, f'Left_Bottom Counter: {tc}', (10, lh + 30), font, size, black, t)
    cv2.putText(frame, f'Right_Bottom Counter: {bc}', (lw + 10, lh + 30), font, size, black, t)

    output.write(frame)


0: 384x640 2 buss, 16 cars, 300.5ms
Speed: 3.9ms preprocess, 300.5ms inference, 1940.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 17 cars, 62.9ms
Speed: 11.1ms preprocess, 62.9ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 cars, 62.9ms
Speed: 3.1ms preprocess, 62.9ms inference, 4.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 cars, 54.6ms
Speed: 5.4ms preprocess, 54.6ms inference, 8.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bus, 17 cars, 53.6ms
Speed: 3.5ms preprocess, 53.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 19 cars, 75.7ms
Speed: 3.3ms preprocess, 75.7ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 cars, 76.9ms
Speed: 3.3ms preprocess, 76.9ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 bus, 17 cars, 53.6ms
Speed: 3.2ms preprocess, 53.6ms inference, 2.2ms postprocess per imag

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

## Save and Submit Your Work

Submit both the notebook and the output video